In [1]:
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
from tensorflow.python.framework import graph_util
from keras.layers import *
from keras.optimizers import *
from sklearn.model_selection import train_test_split
import sys
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt
import cv2
import time
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import Adam
from skimage.io import imshow
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from sklearn.utils import shuffle
import numpy.random as rng

Using TensorFlow backend.
C:\Users\Abhilasha\Anaconda2\envs\tensorflow_env\lib\site-packages\skimage\external\tifffile\tifffile.py:305: UserWarning: ImportError: No module named '_tifffile'. Loading of some compressed images will be very slow. Tifffile.c can be obtained at http://www.lfd.uci.edu/~gohlke/
  "ImportError: No module named '_tifffile'. "


In [2]:
data1='D://fintest//1coal'
data2='D://fintest//2coal'
data3='D://fintest//3coal'

In [3]:
def data_with_label():
    images=[]
    cnt = 0
    for i in tqdm(os.listdir(data1)):
        path=os.path.join(data1,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(64,64))
        images.append([np.array(img),'0'])
    cnt = 0
    for i in tqdm(os.listdir(data2)):
        path=os.path.join(data2,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(64,64))
        images.append([np.array(img),'1'])
    cnt = 0
    for i in tqdm(os.listdir(data3)):
        path=os.path.join(data3,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(64,64))
        images.append([np.array(img),'2'])
        
    return images

In [4]:
data = data_with_label()

100%|████████████████████████████████████████████████████████████████████████████████| 455/455 [00:31<00:00, 12.12it/s]


In [5]:
import random 
random.shuffle(data)
x_train = np.array([i[0] for i in data]).reshape(-1,64,64,1)
x_train = x_train.astype('float32')
x_train /= 255
y_train = np.array([i[1] for i in data])

In [6]:
#we have a total of 1154 images taking 500 to generate valid training pairs
image_list = np.split(x_train[:500],500)
label_list = np.split(y_train[:500],500)

left_input = []
right_input = []
targets = []

#every single image will be paired with 10 dustinct images along with the label 1 if they match else the label 0
pairs = 10
#total number of training pairs generated will be 5000 (500*10)

for i in range(len(label_list)):
    for _ in range(pairs):
        compare_to = i
        while compare_to == i: #to make sure it's not comparing with itself
            compare_to = random.randint(0,499)
        left_input.append(image_list[i])
        right_input.append(image_list[compare_to])
        if label_list[i] == label_list[compare_to]:
            targets.append(1.)
        else:
            targets.append(0.)
            
left_input = np.squeeze(np.array(left_input))
right_input = np.squeeze(np.array(right_input))
targets = np.squeeze(np.array(targets))

#taking a random image from the traing set and finding its correct pairs from the testing set
img1 = x_train[68]
#total number of testing pairs will be (1154-500)*1 thai is 654 pairs
test_left = []
test_right = []
test_targets = []

for i in range(y_train.shape[0]-500):
    test_left.append(img1)
    test_right.append(x_train[i+500])
    if y_train[68]==y_train[i+500]:
        test_targets.append(1.)
    else:
        test_targets.append(0.)

test_left = np.squeeze(np.array(test_left))
test_right = np.squeeze(np.array(test_right))
test_targets = np.squeeze(np.array(test_targets))

In [7]:
test_right = np.array([i for i in test_right]).reshape(-1,64,64,1)
test_left = np.array([i for i in test_left]).reshape(-1,64,64,1)
left_input = np.array([i for i in left_input]).reshape(-1,64,64,1)
right_input = np.array([i for i in right_input]).reshape(-1,64,64,1)
print(x_train.shape)
print(test_right.shape)
print(test_left.shape)
print(left_input.shape)
print(right_input.shape)

(1144, 64, 64, 1)
(644, 64, 64, 1)
(644, 64, 64, 1)
(5000, 64, 64, 1)
(5000, 64, 64, 1)


In [8]:
#taking two images to check their similarity
def create_model():

    left_input = Input((64,64,1))
    right_input = Input((64,64,1))

# Base network architecture

    convnet=Sequential()
    convnet.add(InputLayer(input_shape=[64,64,1]))
    convnet.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same',activation='relu'))
    convnet.add(MaxPool2D(pool_size=5,padding='same'))   
    convnet.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same',activation='relu'))
    convnet.add(MaxPool2D(pool_size=5,padding='same'))  
    convnet.add(Conv2D(filters=64,kernel_size=3,strides=1,padding='same',activation='relu'))
    convnet.add(MaxPool2D(pool_size=8,padding='same'))
    convnet.add(Conv2D(filters=128,kernel_size=3,strides=1,padding='same',activation='relu'))
    convnet.add(MaxPool2D(pool_size=5,padding='same'))   
    convnet.add(Conv2D(filters=256,kernel_size=3,strides=1,padding='same',activation='relu'))
    convnet.add(Flatten())

    convnet.add(Dense(1024,activation='relu'))
    convnet.add(Dropout(0.20))
    convnet.add(Dense(512,activation='relu'))
    convnet.add(Dropout(0.25))
    convnet.add(Dense(64,activation='sigmoid'))

#generating the encodings for two input images
    encoded_l = convnet(left_input)
    encoded_r = convnet(right_input)

# Getting the L1 Distance between the 2 encodings
    L1_layer = Lambda(lambda tensor:K.abs(tensor[0] - tensor[1]))

# Add the distance function to the network
    L1_distance = L1_layer([encoded_l, encoded_r])

    prediction = Dense(1,activation='sigmoid')(L1_distance)
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

    optimizer = Adam(0.0001, decay=2.5e-4)
    siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=['accuracy'])
    return siamese_net

In [9]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate([test_left,test_right], test_targets, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

Untrained model, accuracy: 41.61%


In [10]:
# Loads the weights
checkpoint_path='cpss2.ckpt'
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate([test_left,test_right], test_targets, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Restored model, accuracy: 94.41%


In [26]:
#images from 3 classes 
left= cv2.imread("D://fintest//1coal//pic1crop.jpg", cv2.IMREAD_GRAYSCALE)
left= cv2.resize(left,(64,64))
left=np.array(left).reshape(-1,64,64,1)
left= left/255

left1= cv2.imread("D://fintest//2coal//pic1crop.jpg", cv2.IMREAD_GRAYSCALE)
left1= cv2.resize(left1,(64,64))
left1=np.array(left1).reshape(-1,64,64,1)
left1= left1/255

left2= cv2.imread("D://fintest//3coal//pic1crop.jpg", cv2.IMREAD_GRAYSCALE)
left2= cv2.resize(left2,(64,64))
left2=np.array(left2).reshape(-1,64,64,1)
left2= left2/255

#test image from class 1
right= cv2.imread("D://fintest//1coal//pic7crop.jpg", cv2.IMREAD_GRAYSCALE)
right=cv2.resize(right,(64,64))
right=np.array(right).reshape(-1,64,64,1)
right= right/255


print(model.predict([left,right]))
print(model.predict([left1,right]))
print(model.predict([left2,right]))

[[0.5754501]]
[[0.00064226]]
[[6.975044e-08]]
